<a href="https://colab.research.google.com/github/rodrigocardosodev/run-openweb-ui-google-colab/blob/master/run_openwebui_colab.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Tutorial: Execução do OpenWebUI no Google Colab

Este notebook foi preparado para demonstrar, passo a passo, como configurar e executar o OpenWebUI no ambiente do Google Colab. Em cada etapa, você encontrará uma célula de texto que explica detalhadamente o que está sendo feito e por que cada comando é importante para a execução da aplicação.
      

# Configuração de Variáveis de Ambiente

Antes de iniciar, defina as seguintes variáveis de ambiente:
- **HF_TOKEN:** Token para autenticação no Hugging Face.
- **NGROK_TOKEN:** Token para configuração do ngrok.

Você pode definir essas variáveis utilizando a célula de código abaixo (ou diretamente no ambiente do Colab):

```python
# Exemplo:
# %env HF_TOKEN=seu_token_aqui
# %env NGROK_TOKEN=seu_token_aqui


In [ ]:
%%env HF_TOKEN=
%%env NGROK_TOKEN=


---

### 3. Atualização do Sistema e Instalação de Dependências

```markdown
# Atualização do Sistema e Instalação de Dependências

Nesta etapa, o sistema é atualizado e são instaladas as dependências necessárias:

- **Atualização do sistema:** Atualiza os pacotes do sistema.
- **tmux:** Ferramenta para gerenciar sessões de terminal em background.
- **pip:** Atualiza o gerenciador de pacotes do Python.
- **huggingface_hub e open-webui:** Bibliotecas essenciais para executar o OpenWebUI.
- **ngrok:** Configura um túnel público para expor o serviço.
- **Ollama:** Instala o backend de IA (caso necessário).

Os comandos utilizados estão comentados para garantir a compatibilidade com o Python no Colab.


In [ ]:
%%capture
!apt update && apt upgrade

In [ ]:
%%capture
!apt install tmux -qq

In [ ]:
%%capture
!pip install -U pip -q

In [ ]:
%%capture
!pip install -U huggingface_hub open-webui -q

In [ ]:
%%capture
!curl -sSL https://ngrok-agent.s3.amazonaws.com/ngrok.asc \
  | sudo tee /etc/apt/trusted.gpg.d/ngrok.asc >/dev/null \
  && echo "deb https://ngrok-agent.s3.amazonaws.com buster main" \
  | sudo tee /etc/apt/sources.list.d/ngrok.list \
  && sudo apt update \
  && sudo apt install ngrok

In [ ]:
%%capture
!curl -fsSL https://ollama.com/install.sh | sh

In [ ]:
HF_TOKEN = os.enviroment["HF_TOKEN"] if "HF_TOKEN" in os.enviroment else ""
NGROK_TOKEN = os.enviroment["NGROK_TOKEN"] if "NGROK_TOKEN" in os.enviroment else ""

# Login no Hugging Face

Realizamos o login no Hugging Face utilizando o token definido na variável **HF_TOKEN**. Esse passo é essencial para acessar os recursos e modelos disponíveis na plataforma.

Caso o token não esteja definido corretamente, será exibida a mensagem: "Token não encontrado".


In [ ]:
from huggingface_hub import notebook_login

import os

try:
    notebook_login(token=os.enviroment["HF_TOKEN"])
except:
    print("Token não encontrado")

# Baixar Modelo (Opcional)

Caso seja necessário utilizar um modelo específico com o Ollama, descomente o comando correspondente na célula de código para realizar o download do modelo desejado.

*Observação:* Esta etapa é opcional e deve ser utilizada se o modelo for requerido para a execução da aplicação.


In [ ]:
%%capture
!ollama pull gemma3:27b-it-q8_0

# Configuração do ngrok

Utilize o comando abaixo para adicionar o token de autenticação ao ngrok, permitindo a criação do túnel para o serviço web:




In [ ]:
!ngrok config add-authtoken $NGROK_TOKEN


---

### 7. Início dos Serviços com tmux


**Início dos Serviços com tmux**

Nesta etapa, iniciamos os seguintes serviços em sessões separadas utilizando o tmux:

- **Ollama:** Inicia o serviço backend de IA com o comando `ollama serve`.
- **OpenWebUI:** Lança a interface web do serviço na porta 8081 com o comando `open-webui serve --port 8081`.
- **ngrok:** Cria um túnel para expor a interface web (na porta 8081) à internet com o comando `ngrok http http://localhost:8081`.

Esses comandos garantem que os serviços continuem rodando em background, mesmo que a sessão principal do Colab seja interrompida.


In [ ]:
# Inicie o serviço Ollama em background
!tmux new-session -d -s ollama_session ollama serve

In [ ]:
# Inicie o serviço open-webui em background
!tmux new-session -d -s openweb_session open-webui serve --port 8081

In [ ]:
# Inicie o serviço ngrok em background
!tmux new-session -d -s ngrok_session ngrok http http://localhost:8081

# Verificação das Sessões Ativas do tmux

Por fim, execute o comando abaixo para listar todas as sessões ativas do tmux. Dessa forma, você pode confirmar que as sessões **ollama_session**, **openweb_session** e **ngrok_session** estão rodando corretamente:



In [ ]:
!tmux ls

# Finalizando


Ao final verifique na sua conta do ngrok qual o endpoint que foi exposto pelo google colab.

# Conclusão e Considerações Finais

Este script oferece uma solução prática para rodar o openweb-ui e testar aplicações de IA diretamente no Google Colab, integrando com ferramentas como ngrok e ollama. Ele automatiza a configuração do ambiente, garantindo que todas as dependências sejam instaladas e que os serviços necessários sejam iniciados corretamente.

## Vantagens

- **Simplicidade:** Automatiza várias etapas, eliminando a necessidade de configurações manuais complexas.
- **Execução Estável:** Utiliza o tmux para rodar serviços em background, permitindo que a sessão principal do Colab seja encerrada sem interromper o funcionamento.
- **Integração Eficiente:** Facilita o acesso remoto à interface web por meio do ngrok e integra recursos de IA via Hugging Face e Ollama.

## Desvantagens

- **Configuração Inicial:** Requer a definição prévia de variáveis de ambiente, como os tokens de acesso (HF_TOKEN e NGROK_TOKEN), o que pode ser um obstáculo para iniciantes.
- **Limitações do Colab:** O ambiente do Google Colab possui restrições que podem afetar a continuidade e o desempenho dos serviços.
